## RelationsLayer

RelationsLayer is a new type of layer introduced in estntlk v1.7.2. 
It allows to store information about relationships between entities mentioned in text, such as coreference relations or semantic roles.

### Example 1: coreference relations

In [1]:
from estnltk import Text
from estnltk_core import RelationsLayer

In [2]:
text = Text('Mari kirjeldas õhinal, kuidas ta väiksena "Sipsikut" luges: '+\
'"Ma ei suutnud seda raamatut kohe kuidagi käest ära panna! Nii põnev oli see!"').tag_layer('words')

In [3]:
# Get word span locations:
#text.words[['start', 'end', 'text']]

Upon creating a RelationsLayer, you need to define names for the spans, and names for the attributes.
Span names are required, attribute names can also be skipped:

In [4]:
coref_layer = RelationsLayer('coreference', span_names=['mention', 'entity'], text_object=text)

Use add_annotation method to add new relations to the layer:

In [5]:
# Add relation based on a dictionary
coref_layer.add_annotation( {'mention': (30, 32), 'entity': (0, 4)} )
coref_layer.add_annotation( {'mention': (61, 63), 'entity': (0, 4)} )
# Or add relation by keyword arguments
coref_layer.add_annotation( mention=(75, 88), entity=(42, 52) )
coref_layer.add_annotation( mention=(133, 136), entity=(42, 52) )
coref_layer

RelationsLayer(name='coreference', span_names=('mention', 'entity'), attributes=(), relations=[Relation([NamedSpan(mention: 'ta'), NamedSpan(entity: 'Mari')], [{}]), Relation([NamedSpan(mention: 'Ma'), NamedSpan(entity: 'Mari')], [{}]), Relation([NamedSpan(mention: 'seda raamatut'), NamedSpan(entity: '"Sipsikut"')], [{}]), Relation([NamedSpan(mention: 'see'), NamedSpan(entity: '"Sipsikut"')], [{}])])

In [6]:
# Access span names
coref_layer.span_names

('mention', 'entity')

In [7]:
# use numeric indexes to access relations
coref_layer[0]

Relation([NamedSpan(mention: 'ta'), NamedSpan(entity: 'Mari')], [{}])

In [8]:
coref_layer[0].mention

NamedSpan(mention: 'ta')

In [9]:
coref_layer[0].mention.text

'ta'

In [10]:
coref_layer[0].entity.text

'Mari'

In [11]:
# get all mentions
coref_layer[['mention']]

[[NamedSpan(mention: 'ta')],
 [NamedSpan(mention: 'Ma')],
 [NamedSpan(mention: 'seda raamatut')],
 [NamedSpan(mention: 'see')]]

In [12]:
# get all entities
coref_layer[['entity']]

[[NamedSpan(entity: 'Mari')],
 [NamedSpan(entity: 'Mari')],
 [NamedSpan(entity: '"Sipsikut"')],
 [NamedSpan(entity: '"Sipsikut"')]]

### Example 2: temporal relations

In [13]:
from estnltk import Text
from estnltk_core import RelationsLayer

In [14]:
text = Text('Pühapäeva varahommikul kutsuti politsei Riia mäele. '+\
'Seal oli ühelt noorelt mehelt ära võetud nahktagi ja käekell. '+\
'Juhtumi kohta algatati uurimine.').tag_layer('words')

In [15]:
# Get word span locations:
#text.words[['start', 'end', 'text']]

In [16]:
tlinks_layer = RelationsLayer('temporal_relations', span_names=['entity_a', 'entity_b'], 
                                                    attributes=['rel_type'], text_object=text)

In [17]:
# Access span names
tlinks_layer.span_names

('entity_a', 'entity_b')

In [18]:
# Access attributes
tlinks_layer.attributes

('rel_type',)

In [19]:
# Add relation based on a dictionary
tlinks_layer.add_annotation( {'entity_a': (0, 22),  'entity_b': (23, 30),   'rel_type': 'INCLUDES'} )
tlinks_layer.add_annotation( {'entity_a': (82, 92), 'entity_b': (23, 30),   'rel_type': 'BEFORE'} )
# Or add relation by keyword arguments
tlinks_layer.add_annotation( entity_a=(82, 92), entity_b=(114, 121), rel_type='IDENTITY' )
tlinks_layer.add_annotation( entity_a=(82, 92), entity_b=(128, 145), rel_type='BEFORE' )
tlinks_layer

RelationsLayer(name='temporal_relations', span_names=('entity_a', 'entity_b'), attributes=('rel_type',), relations=[Relation([NamedSpan(entity_a: 'Pühapäeva varahommikul'), NamedSpan(entity_b: 'kutsuti')], [{'rel_type': 'INCLUDES'}]), Relation([NamedSpan(entity_a: 'ära võetud'), NamedSpan(entity_b: 'kutsuti')], [{'rel_type': 'BEFORE'}]), Relation([NamedSpan(entity_a: 'ära võetud'), NamedSpan(entity_b: 'Juhtumi')], [{'rel_type': 'IDENTITY'}]), Relation([NamedSpan(entity_a: 'ära võetud'), NamedSpan(entity_b: 'algatati uurimine')], [{'rel_type': 'BEFORE'}])])

In [20]:
tlinks_layer[0]

Relation([NamedSpan(entity_a: 'Pühapäeva varahommikul'), NamedSpan(entity_b: 'kutsuti')], [{'rel_type': 'INCLUDES'}])

In [21]:
tlinks_layer[0]['rel_type']

'INCLUDES'

### Example 3: semantic roles

In [22]:
from estnltk import Text
from estnltk_core import RelationsLayer

In [23]:
text = Text('President Bush kohtus temaga privaatselt Valges Majas teisipäeval. '+\
'Aga John ja Mari kohtusid hoopis kokteilipeol. '+\
'Mari ei ostnud Johnile kokteili.').tag_layer('words')

In [24]:
# Get word span locations:
#text.words[['start', 'end', 'text']]

You can also define more span names than only the minimal set to be used in every relation. 
Some of the named spans can be filled out only in specific contexts. 
Next, we follow the example of English PropBank guidelines and define a layer of semantic roles, which have slots/placeholders for different arguments. However, whether an argument is realized or not, depends on the context:

In [25]:
sem_roles_layer = RelationsLayer('semantic_roles', span_names=['arg0', 'arg1', 'arg2', 'arg3', 
                                                               'arg4', 'argm_mnr', 'argm_tmp', 
                                                               'argm_loc'], 
                                                   attributes=['rel'], text_object=text)
# Based on PropBank English guidelines:
# ARG0 -- agent
# ARG1 -- patient 
# ARG2 -- instrument, benefactive, attribute 
# ARG3 -- starting point, benefactive, attribute
# ARG4 -- ending point, beneficiary
# ARGM -- modifier (manner, time, location)

In [26]:
sem_roles_layer.add_annotation( {'arg0': (0, 14),  'arg1': (22, 28), 'argm_mnr': (29, 40), 
                                 'argm_loc': (41,53), 'argm_tmp': (54, 65), 'rel': 'kohtumine'} )
sem_roles_layer.add_annotation( {'arg0': (71, 75), 'arg1': (79, 83), 'rel': 'kohtumine'} )
sem_roles_layer.add_annotation( {'arg0': (114, 118), 'arg1': (137, 145), 'arg4': (129, 136), 
                                 'rel': 'ostmine-NEG'} )
sem_roles_layer

RelationsLayer(name='semantic_roles', span_names=('arg0', 'arg1', 'arg2', 'arg3', 'arg4', 'argm_mnr', 'argm_tmp', 'argm_loc'), attributes=('rel',), relations=[Relation([NamedSpan(arg0: 'President Bush'), NamedSpan(arg1: 'temaga'), NamedSpan(argm_mnr: 'privaatselt'), NamedSpan(argm_tmp: 'teisipäeval'), NamedSpan(argm_loc: 'Valges Majas')], [{'rel': 'kohtumine'}]), Relation([NamedSpan(arg0: 'John'), NamedSpan(arg1: 'Mari')], [{'rel': 'kohtumine'}]), Relation([NamedSpan(arg0: 'Mari'), NamedSpan(arg1: 'kokteili'), NamedSpan(arg4: 'Johnile')], [{'rel': 'ostmine-NEG'}])])

In [27]:
sem_roles_layer[0]

Relation([NamedSpan(arg0: 'President Bush'), NamedSpan(arg1: 'temaga'), NamedSpan(argm_mnr: 'privaatselt'), NamedSpan(argm_tmp: 'teisipäeval'), NamedSpan(argm_loc: 'Valges Majas')], [{'rel': 'kohtumine'}])

In [28]:
sem_roles_layer[0].spans

[NamedSpan(arg0: 'President Bush'),
 NamedSpan(arg1: 'temaga'),
 NamedSpan(argm_mnr: 'privaatselt'),
 NamedSpan(argm_tmp: 'teisipäeval'),
 NamedSpan(argm_loc: 'Valges Majas')]